# Renault car sequencing I -- Exploration

Herkunft von Problem und Daten:
https://www.roadef.org/challenge/2005/en/


Es werden fünf Pakete importiert. 
<ul>
    <li><code>pandas</code>: Manipulation von Tabellendaten</li>
    <li><code>numpy</code>: wissenschaftliche Bibliothek, im wesentlichen für die array-Klasse</li>
    <li><code>matplotlib</code>: Visualisierung</li>
    <li><code>os</code>: Systemzugriffen, insbesondere Navigation des Dateisystems</li>
    <li><code>re</code>: regular expressions</li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import math
import random

In [3]:
path="../RENAULT/Instances_set_A/"   # Pfad zu den gespeicherten Instanzen -- ersetzen Sie das durch den entsprechenden Pfad bei Ihnen
#!wget https://www.roadef.org/challenge/2005/files/Instances_set_A.zip
#!unzip Instances_set_A.zip
#path="Instances/"


## Methoden zum Einlesen aller Instanzen

Die Daten bestehen aus eine Reihe von Instanzen. Jede Instanz umfasst die Produktion an einem Tag in einer Fertigungsstätte. Renault verfügt über mehrere Fertigungsstätten. Entsprechend können sich die Instanzen unterscheiden.
Alle Instanzen unter path werden in ein dictionary eingelesen <code>data_dict</code>. Den Code in der folgenden Zelle müssen Sie nicht genauer angucken. Einzig die Klasse <code>Vehicle</code> ist interessant.

In [5]:
class Vehicle:
    def __init__(self,ident,colour,options):
        self.ident=ident
        self.colour=colour
        self.options=options
        
    def __repr__(self):
        return str(self.ident)
    
    def __str__(self):
        return "id: {}, Farbe: {}, Optionen: {}".format(self.ident,self.colour,self.options)

def get_weights(opt_objectives):
    weights={'paint':0, 'high':0, 'low':0}
    w=[1000000,1000,1]
    for i,row in opt_objectives.iterrows():
        key=re.match(r"([a-z]+)_",row['objective name']).group(1)
        weights[key]=w[row['rank']-1]
    return weights
    
def read_objectives(filepath):
    opt_objectives=pd.read_csv(filepath,sep=';')
    opt_objectives=opt_objectives.drop(columns="Unnamed: 2")
    return get_weights(opt_objectives)
    
def parse_paint_batch_limit(filepath):
    paint_batch_limit_csv=pd.read_csv(filepath,sep=';')
    paint_batch_limit=paint_batch_limit_csv.loc[0,'limitation']
    return paint_batch_limit    

def read_ratios(filepath):
    ratios_table=pd.read_csv(filepath,sep=';')
    ratios_table['p']=ratios_table["Ratio"].str.extract(r"(\d*)/").astype('int')
    ratios_table['q']=ratios_table["Ratio"].str.extract(r"/(\d*)").astype('int')
    ratios_table=ratios_table.drop(columns=['Unnamed: 3'])
    ratios={}
    for i,row in ratios_table.iterrows():
        ratios[row['Ident']]=(row['p'],row['q'])
    return ratios

def process_vehicles(vehicles):
    previous_day_vehicles=[]
    current_day_vehicles=[]
    for i,row in vehicles.iterrows():
        options=[option for option in row[4:-1].index if row[option]==1]
        car=Vehicle(row["Ident"],row['Paint Color'],options)
        if row['already scheduled']:
            previous_day_vehicles.append(car)
        else:
            current_day_vehicles.append(car)
    return previous_day_vehicles,current_day_vehicles

def read_vehicles(filepath):
    vehicles=pd.read_csv(filepath,sep=';')
    previous_day=vehicles['Date'].iloc[0]
    vehicles["already scheduled"]=vehicles['Date']==previous_day
    return vehicles

def get_renault_schedule(vehicles):
    """
    Die Reihung des Renault-Algorithmus
    """
    renault_schedule=list(vehicles.loc[vehicles['already scheduled']==False,'Ident'])
    return renault_schedule

def read_in_all_instances(path,silent=False):
    data_dict={}
    for root, dirs, files in os.walk(path):
        instance_dict={}
        rest,first=os.path.split(root)
        data_dict[first]=instance_dict
        for filename in files:
            filepath=os.path.join(root,filename)
            if filename=="optimization_objectives.txt":
                opt_obj=read_objectives(filepath)
                instance_dict["weights"]=opt_obj            
            if filename=="paint_batch_limit.txt":
                paint_batch_limit=parse_paint_batch_limit(filepath)
                instance_dict['paint_batch_limit']=paint_batch_limit
            if filename=="ratios.txt":
                ratios=read_ratios(filepath)
                instance_dict["ratios"]=ratios            
            if filename=="vehicles.txt":
                vehicles=read_vehicles(filepath)
                previous,current=process_vehicles(vehicles)
                #instance_dict["previous_day"]=previous   # we don't use this
                instance_dict["current_day"]=current
                instance_dict["renault_schedule"]=current.copy()
    delete=[]
    for key in data_dict.keys():
        if data_dict[key]=={}:
            delete.append(key)
        data_dict[key]['name']=key
    for key in delete:
        del data_dict[key]
    if not silent:
        print("Folgende Instanzen wurden eingelesen: ")
        for key in data_dict.keys():
            print("  "+key)
    return data_dict

from operator import itemgetter
def prio_string(instance):
    prio=[what for what,weight in sorted(sample_instance['weights'].items(),key=itemgetter(1),reverse=True)]
    return "{} >> {} >> {}".format(prio[0],prio[1],prio[2])

Wir lesen alle Instanzen ein und speichern sie in einem dictionary. 

In [6]:
data_dict=read_in_all_instances(path)

Folgende Instanzen wurden eingelesen: 
  039_38_4_EP_RAF_ch1
  024_38_3_EP_ENP_RAF
  048_39_1_EP_ENP_RAF
  025_38_1_EP_ENP_RAF
  064_38_2_EP_RAF_ENP_ch1
  064_38_2_RAF_EP_ENP_ch2
  022_3_4_EP_RAF_ENP
  048_39_1_EP_RAF_ENP
  024_38_5_EP_RAF_ENP
  039_38_4_RAF_EP_ch1
  024_38_3_EP_RAF_ENP
  025_38_1_EP_RAF_ENP
  064_38_2_EP_RAF_ENP_ch2
  064_38_2_RAF_EP_ENP_ch1
  022_3_4_RAF_EP_ENP
  024_38_5_EP_ENP_RAF


Wir gucken uns nun eine Instanz an. Jede Instanz ist wiederum ein dictionary und hält verschiedene Daten bereit. Mit <code>dictionary.keys()</code> erhalten wir eine Liste der im dictionary abgespeicherten Schlüsselwörter.

In [7]:
sample_instance=data_dict['064_38_2_EP_RAF_ENP_ch1']
sample_instance.keys()

dict_keys(['ratios', 'weights', 'current_day', 'renault_schedule', 'paint_batch_limit', 'name'])

In [8]:
sample_instance['name']

'064_38_2_EP_RAF_ENP_ch1'

In [9]:
sample_instance['current_day'][:10]  ## nur die ersten 10 Einträge

[64033840771,
 64033750048,
 64033820923,
 64033820874,
 64033810649,
 64033820888,
 64033820985,
 64033806803,
 64033840889,
 64033810753]

Was wir sehen: die Id-Nummern der Fahrzeuge. In der Liste sind aber tatsächlich die Fahrzeuge gespeichert, und zwar als Objekte. Betrachten wir ein Fahrzeug.

In [10]:
car=sample_instance['current_day'][42]
print("Fahrzeug Id-Nr {} hat Farbe {}".format(car.ident,car.colour))
print("Es sollen folgende Optionen verbaut werden: {}".format(car.options))

Fahrzeug Id-Nr 64033840854 hat Farbe 7
Es sollen folgende Optionen verbaut werden: ['HPRC5', 'LPRC2']


Dh jeder Eintrag <code>car</code> in der Liste <code>sample_instance['current_day']</code> hat die Eigenschaften
<ul>
    <li><code>car.ident</code>: die Id-Nr</li>
    <li><code>car.colour</code>: die Farbe</li>
    <li><code>car.options</code>: die zu verbauenden Optionen</li>
</ul>

Mit <code>print</code> können diese Eigenschaft direkt ausgegeben werden.

In [11]:
print(sample_instance['current_day'][6])

id: 64033820985, Farbe: 1, Optionen: ['HPRC7', 'LPRC2']


Weiterhin enthält die Instanz Information zu den $p/q$-Bedinungen, abgespeichert als dictionary. Die Einträge sind jeweils als <code>(p,q)</code> abgelegt.

In [12]:
sample_instance['ratios']

{'HPRC1': (1, 8),
 'HPRC2': (1, 2),
 'HPRC3': (1, 7),
 'HPRC4': (1, 15),
 'HPRC5': (2, 3),
 'HPRC6': (1, 30),
 'HPRC7': (4, 5),
 'LPRC1': (1, 6),
 'LPRC2': (1, 3)}

Renault hat jeweils bereits eine Reihung algorithmisch ermittelt. Zum Vergleich können wir sie heranziehen. Wir wollen natürlich besser sein.

In [13]:
sample_instance['renault_schedule'][:10]  ## wir zeigen nur die ersten 10 Einträge an

[64033840771,
 64033750048,
 64033820923,
 64033820874,
 64033810649,
 64033820888,
 64033820985,
 64033806803,
 64033840889,
 64033810753]

Es folgen zwei weitere Parameter: das paint_batch_limit und die Gewichtung der Ziele

In [14]:
sample_instance['paint_batch_limit']

15

In [15]:
sample_instance['weights']

{'paint': 1000, 'high': 1000000, 'low': 1}

Dies ist so zu lesen: Größte Priorität bei der Optimierung wird den high-priority Optionen zugewiesen, dann folgt die Zahl der Farbwechsel und schließlich die low-priority Optionen. 

Um das Ziel der Optimierung besser für den/die Anwender/in zu verdeutlichen, gibt's auch noch eine Methode:

In [16]:
prio_string(sample_instance)

'high >> paint >> low'

## 1. Aufgabe: Exploration der Instanzen

Bevor der erste Algorithmus formuliert wird, sollten die Daten gesichtet werden. Manchmal ergibt sich die eine oder andere Überraschung: ein Aspekt des Problems ist anders als vom Anwender dargestellt, andere Aspekte erweisen sich als unwichtig, da sie kaum vorkommen usw. Wichtig ist es auch, eine Idee vom Umfang des Problems zu erhalten. Wie groß sind überhaupt die Instanzen?

Überlegen Sie sich, welche Daten Sie über die Instanzen erheben wollen. Betrachten Sie einzelne Instanzen und aber auch die Gesamtheit der Instanzen. Welche Daten können Ihnen Einblicke in das Problem verschaffen? Beispielhaft listen wir als erstes die Instanzgrößen auf, dh, wie viele Fahrzeuge jeweils produziert werden sollen:

In [17]:
sizes=[len(instance['current_day']) for instance in data_dict.values()]
sorted(sizes)

[335,
 335,
 485,
 485,
 600,
 600,
 875,
 875,
 954,
 954,
 1004,
 1004,
 1260,
 1260,
 1315,
 1315]

Und jetzt Sie! 